张吉祺 16340286

**We want to build a naïve bayes sentiment classifier using add -1 smoothing, as described in the lecture (not binary naïve bayes, regular naïve bayes). Here is our training corpus:**

**Training Set:**

\- just plain boring  
\- entirely predictable and lacks energy  
\- no surprises and very few laughs  
\+ very powerful  
\+ the most fun film of the summer

**Test Set:**

predictable with no originality

In [1]:
train_neg = [
    'just plain boring',
    'entirely predictable and lacks energy',
    'no surprises and very few laughs'
]
train_pos = [
    'very powerful',
    'the most fun film of the summer'
]
test = 'predictable with no originality'

V = set()
def mk_dict_and_cnt(sentences):
    res = {}
    cnt = 0
    for i in sentences:
        for j in i.split():
            if j in res:
                res[j] += 1
            else:
                res[j] = 1
            cnt += 1
            V.add(j)
    return res, cnt

neg_dict, neg_cnt = mk_dict_and_cnt(train_neg)
pos_dict, pos_cnt = mk_dict_and_cnt(train_pos)

**1. Compute the prior for the two classes + and -, and the likelihoods for each word given the class (leave in the form of fractions).**

In [2]:
V_size = len(V)
neg_pri = len(train_neg) / (len(train_neg) + len(train_pos))
print('P(-)\t\t= {}/{}'.format(len(train_neg), (len(train_neg) + len(train_pos))))
pos_pri = len(train_pos) / (len(train_neg) + len(train_pos))
print('P(+)\t\t= {}/{}'.format(len(train_pos), (len(train_neg) + len(train_pos))))
for k, v in neg_dict.items():
    print('P(\'{}\'|-)\t= {}/{}'.format(k, v + 1, neg_cnt + V_size))
print('P(word not in -|-)\t= {}/{}'.format(1, neg_cnt + V_size))
for k, v in pos_dict.items():
    print('P(\'{}\'|+)\t= {}/{}'.format(k, v + 1, pos_cnt + V_size))
print('P(word not in +|+)\t= {}/{}'.format(1, pos_cnt + V_size))

P(-)		= 3/5
P(+)		= 2/5
P('just'|-)	= 2/34
P('plain'|-)	= 2/34
P('boring'|-)	= 2/34
P('entirely'|-)	= 2/34
P('predictable'|-)	= 2/34
P('and'|-)	= 3/34
P('lacks'|-)	= 2/34
P('energy'|-)	= 2/34
P('no'|-)	= 2/34
P('surprises'|-)	= 2/34
P('very'|-)	= 2/34
P('few'|-)	= 2/34
P('laughs'|-)	= 2/34
P(word not in -|-)	= 1/34
P('very'|+)	= 2/29
P('powerful'|+)	= 2/29
P('the'|+)	= 3/29
P('most'|+)	= 2/29
P('fun'|+)	= 2/29
P('film'|+)	= 2/29
P('of'|+)	= 2/29
P('summer'|+)	= 2/29
P(word not in +|+)	= 1/29


**2. Then compute whether the sentence in the test set is of class positive or negative (you may need a computer for this final computation).**

In [3]:
def posterior(class_prior, class_dict, class_cnt, sentence):
    words = sentence.split()
    res = class_prior
    for i in words:
        if i in V:
            p = class_dict[i] if i in class_dict else 0
            res *= (p + 1) / (class_cnt + V_size)
    return res

neg_post = posterior(neg_pri, neg_dict, neg_cnt, test)
print('P(-|\"{}\")\t='.format(test), neg_post)
pos_post = posterior(pos_pri, pos_dict, pos_cnt, test)
print('P(+|\"{}\")\t='.format(test), pos_post)
if neg_post > pos_post:
    print('P(-|\"{}\") is greater, so the test set sentence is classified as class {}.'.format(test, 'negative'))
elif neg_post < pos_post:
    print('P(+|\"{}\") is greater, so the test set sentence is classified as class {}.'.format(test, 'positive'))

P(-|"predictable with no originality")	= 0.0020761245674740486
P(+|"predictable with no originality")	= 0.0004756242568370987
P(-|"predictable with no originality") is greater, so the test set sentence is classified as class negative.


**3. Would using binary multinomial Naïve Bayes change anything?**

In [4]:
def posterior_bin(class_prior, class_dict, class_cnt, sentence):
    words = sentence.split()
    res = class_prior
    for i in words:
        if i in V:
            p = 1 if i in class_dict else 0
            res *= (p + 1) / (class_cnt + V_size)
    return res

print('Using binary Naïve Bayes:')
neg_post = posterior_bin(neg_pri, neg_dict, len(neg_dict), test)
print('P(-|\"{}\")\t='.format(test), neg_post)
pos_post = posterior_bin(pos_pri, pos_dict, len(pos_dict), test)
print('P(+|\"{}\")\t='.format(test), pos_post)
if neg_post > pos_post:
    print('P(-|\"{}\") is greater, so the test set sentence is classified as class {}.'.format(test, 'negative'))
elif neg_post < pos_post:
    print('P(+|\"{}\") is greater, so the test set sentence is classified as class {}.'.format(test, 'positive'))

Using binary Naïve Bayes:
P(-|"predictable with no originality")	= 0.0022038567493112946
P(+|"predictable with no originality")	= 0.000510204081632653
P(-|"predictable with no originality") is greater, so the test set sentence is classified as class negative.


**4. Why do you add |𝑉| to the denominator of add-1 smoothing, instead of just counting the words in one class?**

In add-1 smoothing we assume we have seen each word once regardless of whether they appear in the original class or not and thus add |𝑉| to the denominator. Note that words that do not appear in the train set are 'unk' and are not included in the vocab.

**5. What would the answer to question 2 be without add-1 smoothing?**

P(c|"predictable with no originality") = 0 for class positive because at least one of the words in the test set does not appear in the positive train examples; P('predictable'|+) = P('no'|+) = 0. 